In [1]:
# Finnhub Setup
import requests
import pandas as pd
import datetime
import time

FINNHUB_API_KEY = "API key"  # Replace with your actual key

def get_finnhub_data(symbol, resolution="D", from_days=365):
    end_time = int(time.time())
    start_time = int(end_time - from_days * 86400)

    url = "https://finnhub.io/api/v1/stock/candle"
    params = {
        "symbol": symbol,
        "resolution": resolution,
        "from": start_time,
        "to": end_time,
        "token": FINNHUB_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data.get("s") != "ok":
        print(f"Error fetching data for {symbol}: {data}")
        return pd.DataFrame()

    df = pd.DataFrame({
        "timestamp": pd.to_datetime(data["t"], unit="s"),
        "open": data["o"],
        "high": data["h"],
        "low": data["l"],
        "close": data["c"],
        "volume": data["v"]
    }).set_index("timestamp")

    return df


In [2]:
# ✅ Get tickers for all S&P 500 stocks from Wikipedia
import requests
import pandas as pd

def tickers_sp500():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    html = requests.get(url).text
    df = pd.read_html(html, header=0)[0]
    tickers = df['Symbol'].dropna().tolist()
    return tickers

tickers = tickers_sp500()
print(f"✅ Loaded {len(tickers)} S&P 500 tickers")


C:\Users\930140\AppData\Local\Temp\1/ipykernel_32196/397156897.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html, header=0)[0]


✅ Loaded 503 S&P 500 tickers


In [3]:
# Index return using Finnhub (SPY as proxy for S&P 500)
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

index_symbol = "SPY"
index_df = get_finnhub_data(index_symbol)
index_df = index_df.loc[start_date:end_date]
index_df["close"] = index_df["close"].astype(float)
index_df["Percent Change"] = index_df["close"].pct_change()
index_return = (index_df["Percent Change"] + 1).cumprod().iloc[-1]
print(f"S&P 500 proxy return (SPY): {index_return:.2f}x")


S&P 500 proxy return (SPY): 1.11x


In [4]:
# RS Rating scanner 
returns_multiples = []
failed_tickers = []
successful_tickers = []

for symbol in tickers:
    try:
        df = get_finnhub_data(symbol)
        if df.empty or "close" not in df.columns:
            print(f"⚠️ Skipping {symbol}: No data or bad format")
            failed_tickers.append(symbol)
            returns_multiples.append(0)
            continue

        df["close"] = df["close"].astype(float)
        df = df.loc[start_date:end_date]
        df["Percent Change"] = df["close"].pct_change()
        stock_return = (df["Percent Change"] + 1).cumprod().iloc[-1]
        rs_score = stock_return / index_return
        returns_multiples.append(rs_score)
        successful_tickers.append(symbol)
        print(f"✅ {symbol} processed successfully – RS Score: {rs_score:.2f}")
        
    except Exception as e:
        print(f"❌ Error for {symbol}: {e}")
        failed_tickers.append(symbol)
        returns_multiples.append(0)

# ✅ Save failed tickers to a text file
with open("failed_tickers.txt", "w") as f:
    for ticker in failed_tickers:
        f.write(f"{ticker}\n")

# ✅ Save successful tickers too (optional)
with open("successful_tickers.txt", "w") as f:
    for ticker in successful_tickers:
        f.write(f"{ticker}\n")

print(f"\n📄 {len(successful_tickers)} successfull logged to successfull_tickers.txt")
print(f"📄 {len(failed_tickers)} failed tickers logged to failed_tickers.txt")


✅ MMM processed successfully – RS Score: 1.27
✅ AOS processed successfully – RS Score: 0.72
✅ ABT processed successfully – RS Score: 1.10
✅ ABBV processed successfully – RS Score: 0.97
✅ ACN processed successfully – RS Score: 0.94
✅ ADBE processed successfully – RS Score: 0.74
✅ AMD processed successfully – RS Score: 0.65
✅ AES processed successfully – RS Score: 0.53
✅ AFL processed successfully – RS Score: 1.07
✅ A processed successfully – RS Score: 0.65
✅ APD processed successfully – RS Score: 0.95
✅ ABNB processed successfully – RS Score: 0.84
✅ AKAM processed successfully – RS Score: 0.72
✅ ALB processed successfully – RS Score: 0.43
✅ ARE processed successfully – RS Score: 0.51
✅ ALGN processed successfully – RS Score: 0.62
✅ ALLE processed successfully – RS Score: 1.01
✅ LNT processed successfully – RS Score: 1.05
✅ ALL processed successfully – RS Score: 1.07
✅ GOOGL processed successfully – RS Score: 0.85
✅ GOOG processed successfully – RS Score: 0.86
✅ MO processed successfully

Error fetching data for DASH: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DASH: No data or bad format
Error fetching data for DOV: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DOV: No data or bad format
Error fetching data for DOW: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DOW: No data or bad format
Error fetching data for DHI: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DHI: No data or bad format
Error fetching data for DTE: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DTE: No data or bad format
Error fetching data for DUK: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DUK: No data or bad format
Error fetching data for DD: {'error': 'API limit reached. Please try again later. Remaining Limit: 0'}
⚠️ Skipping DD: No data or bad format

✅ K processed successfully – RS Score: 1.18
✅ KVUE processed successfully – RS Score: 1.01
✅ KDP processed successfully – RS Score: 0.86
✅ KEY processed successfully – RS Score: 0.97
✅ KEYS processed successfully – RS Score: 0.94
✅ KMB processed successfully – RS Score: 0.89
✅ KIM processed successfully – RS Score: 0.99
✅ KMI processed successfully – RS Score: 1.26
✅ KKR processed successfully – RS Score: 1.11
✅ KLAC processed successfully – RS Score: 0.97
✅ KHC processed successfully – RS Score: 0.68
✅ KR processed successfully – RS Score: 1.10
✅ LHX processed successfully – RS Score: 0.89
✅ LH processed successfully – RS Score: 1.02
✅ LRCX processed successfully – RS Score: 0.81
✅ LW processed successfully – RS Score: 0.53
✅ LVS processed successfully – RS Score: 0.82
✅ LDOS processed successfully – RS Score: 0.93
✅ LEN processed successfully – RS Score: 0.59
✅ LII processed successfully – RS Score: 1.08
✅ LLY processed successfully – RS Score: 0.83
✅ LIN processed successfully – RS 

✅ TKO processed successfully – RS Score: 1.39
✅ TSCO processed successfully – RS Score: 0.84
✅ TT processed successfully – RS Score: 1.16
✅ TDG processed successfully – RS Score: 0.97
✅ TRV processed successfully – RS Score: 1.08
✅ TRMB processed successfully – RS Score: 1.12
✅ TFC processed successfully – RS Score: 0.92
✅ TYL processed successfully – RS Score: 1.04
✅ TSN processed successfully – RS Score: 0.80
✅ USB processed successfully – RS Score: 0.96
✅ UBER processed successfully – RS Score: 1.23
✅ UDR processed successfully – RS Score: 0.94
✅ ULTA processed successfully – RS Score: 0.92
✅ UNP processed successfully – RS Score: 0.84
✅ UAL processed successfully – RS Score: 1.30
✅ UPS processed successfully – RS Score: 0.60
✅ URI processed successfully – RS Score: 0.94
✅ UNH processed successfully – RS Score: 0.53
✅ UHS processed successfully – RS Score: 0.94
✅ VLO processed successfully – RS Score: 0.76
✅ VTR processed successfully – RS Score: 1.18
✅ VLTO processed successfully –

In [5]:
# 📌 Variables
tickers = tickers_sp500()
index_name = '^SPY'  # S&P 500 proxy
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

detailedExportList = pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
returns_multiples = []
failed_tickers = []
successful_tickers = []

# 📈 RS Rating Scanner
for symbol in tickers:
    try:
        df = get_finnhub_data(symbol)
        if df.empty or "close" not in df.columns:
            print(f"⚠️ Skipping {symbol}: No data or bad format")
            failed_tickers.append(symbol)
            returns_multiples.append(0)
            continue

        df["close"] = df["close"].astype(float)
        df = df.loc[start_date:end_date]
        df["Percent Change"] = df["close"].pct_change()
        stock_return = (df["Percent Change"] + 1).cumprod().iloc[-1]
        rs_score = stock_return / index_return
        returns_multiples.append(rs_score)
        successful_tickers.append(symbol)
        print(f"✅ {symbol} processed successfully – RS Score: {rs_score:.2f}")
        
    except Exception as e:
        print(f"❌ Error for {symbol}: {e}")
        failed_tickers.append(symbol)
        returns_multiples.append(0)

# 💾 Save tickers and results
pd.DataFrame(tickers, columns=["Ticker"]).to_csv("tickers_sp500.csv", index=False)

with open("failed_tickers.txt", "w") as f:
    f.writelines(f"{t}\n" for t in failed_tickers)

with open("successful_tickers.txt", "w") as f:
    f.writelines(f"{t}\n" for t in successful_tickers)

print(f"\n📄 {len(successful_tickers)} tickers processed successfully.")
print(f"📄 {len(failed_tickers)} failed tickers logged to failed_tickers.txt")

# 📊 Create RS Rating DataFrame
rs_df = pd.DataFrame(list(zip(tickers, returns_multiples)), columns=['Ticker', 'Returns_multiple'])
rs_df['RS_Rating'] = rs_df['Returns_multiple'].rank(pct=True) * 100
rs_df.to_csv("returns_multiples.csv", index=False)

# ✅ Filter top 30% by RS Rating
rs_df = rs_df[rs_df['RS_Rating'] >= rs_df['RS_Rating'].quantile(0.0)]

# ✅ Export all RS-rated stocks without filtering
rs_df.to_csv("rs_full_list.csv", index=False)
print(f"📊 Full RS stock list: {len(rs_df)} saved to rs_full_list.csv")


C:\Users\930140\AppData\Local\Temp\1/ipykernel_32196/397156897.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html, header=0)[0]


✅ MMM processed successfully – RS Score: 1.27
✅ AOS processed successfully – RS Score: 0.72
✅ ABT processed successfully – RS Score: 1.10
✅ ABBV processed successfully – RS Score: 0.97
✅ ACN processed successfully – RS Score: 0.94
✅ ADBE processed successfully – RS Score: 0.74
✅ AMD processed successfully – RS Score: 0.65
✅ AES processed successfully – RS Score: 0.53
✅ AFL processed successfully – RS Score: 1.07
✅ A processed successfully – RS Score: 0.65
✅ APD processed successfully – RS Score: 0.95
✅ ABNB processed successfully – RS Score: 0.84
✅ AKAM processed successfully – RS Score: 0.72
✅ ALB processed successfully – RS Score: 0.43
✅ ARE processed successfully – RS Score: 0.51
✅ ALGN processed successfully – RS Score: 0.62
✅ ALLE processed successfully – RS Score: 1.01
✅ LNT processed successfully – RS Score: 1.05
✅ ALL processed successfully – RS Score: 1.07
✅ GOOGL processed successfully – RS Score: 0.85
✅ GOOG processed successfully – RS Score: 0.86
✅ MO processed successfully

✅ EG processed successfully – RS Score: 0.79
✅ EVRG processed successfully – RS Score: 1.04
✅ ES processed successfully – RS Score: 0.90
✅ EXC processed successfully – RS Score: 0.98
✅ EXE processed successfully – RS Score: 1.22
✅ EXPE processed successfully – RS Score: 1.34
✅ EXPD processed successfully – RS Score: 0.87
✅ EXR processed successfully – RS Score: 0.87
✅ XOM processed successfully – RS Score: 0.83
✅ FFIV processed successfully – RS Score: 1.46
✅ FDS processed successfully – RS Score: 0.93
✅ FICO processed successfully – RS Score: 1.36
✅ FAST processed successfully – RS Score: 1.08
✅ FRT processed successfully – RS Score: 0.84
✅ FDX processed successfully – RS Score: 0.81
✅ FIS processed successfully – RS Score: 0.91
✅ FITB processed successfully – RS Score: 0.92
✅ FSLR processed successfully – RS Score: 0.89
✅ FE processed successfully – RS Score: 0.92
✅ FI processed successfully – RS Score: 1.12
✅ F processed successfully – RS Score: 0.77
✅ FTNT processed successfully – 

✅ ORCL processed successfully – RS Score: 1.20
✅ OTIS processed successfully – RS Score: 0.89
✅ PCAR processed successfully – RS Score: 0.80
✅ PKG processed successfully – RS Score: 0.95
✅ PLTR processed successfully – RS Score: 5.41
✅ PANW processed successfully – RS Score: 1.08
✅ PARA processed successfully – RS Score: 0.84
✅ PH processed successfully – RS Score: 1.13
✅ PAYX processed successfully – RS Score: 1.09
✅ PAYC processed successfully – RS Score: 1.28
✅ PYPL processed successfully – RS Score: 1.02
✅ PNR processed successfully – RS Score: 1.05
✅ PEP processed successfully – RS Score: 0.63
✅ PFE processed successfully – RS Score: 0.69
✅ PCG processed successfully – RS Score: 0.83
✅ PM processed successfully – RS Score: 1.47
✅ PSX processed successfully – RS Score: 0.77
✅ PNW processed successfully – RS Score: 1.02
✅ PNC processed successfully – RS Score: 0.99
✅ POOL processed successfully – RS Score: 0.75
✅ PPG processed successfully – RS Score: 0.75
✅ PPL processed successful

In [6]:
# Checking Minervini conditions of all stocks in given list (DYNAMIC)
# Initialize export list
detailedExportList = pd.DataFrame(columns=[
    'Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", 
    "52 Week Low", "52 week High"
])

# ✅ Use only the top 30% RS stocks
rs_stocks = rs_df['Ticker']

for stock in rs_stocks:
    try:
        # Fetch data from Finnhub
        df = get_finnhub_data(stock, from_days=500)
        if df.empty:
            raise ValueError("No data returned")

        # Clean and prep columns
        df.columns = [col.split(". ")[-1] for col in df.columns]
        df["close"] = df["close"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)

        # Compute SMAs
        df["SMA_50"] = df["close"].rolling(window=50).mean()
        df["SMA_150"] = df["close"].rolling(window=150).mean()
        df["SMA_200"] = df["close"].rolling(window=200).mean()

        # Grab latest values
        currentClose = df["close"].iloc[-1]
        moving_average_50 = df["SMA_50"].iloc[-1]
        moving_average_150 = df["SMA_150"].iloc[-1]
        moving_average_200 = df["SMA_200"].iloc[-1]
        
        # Check for missing SMA values
        # Check for missing SMA values
        if pd.isnull([moving_average_50, moving_average_150, moving_average_200]).any():
            print(f"📉 Data too short for {stock}: {len(df)} rows")
            status_msg = (f"❌ Skipping {stock}: Missing SMA values - "
                          f"SMA_50: {moving_average_50}, "
                          f"SMA_150: {moving_average_150}, "
                          f"SMA_200: {moving_average_200}")
    
            detailedExportList = pd.concat([detailedExportList, pd.DataFrame([{
                'Stock': stock, 
                "RS_Rating": RS_Rating,
                "50 Day MA": moving_average_50, 
                "150 Day Ma": moving_average_150, 
                "200 Day MA": moving_average_200, 
                "52 Week Low": low_of_52week, 
                "52 week High": high_of_52week,
                "Status": status_msg
            }])], ignore_index=True)

            print(status_msg)
            continue
         
        low_of_52week = round(df["low"].iloc[-260:].min(), 2)
        high_of_52week = round(df["high"].iloc[-260:].max(), 2)
        RS_Rating = round(rs_df[rs_df['Ticker'] == stock].RS_Rating.iloc[0])

        try:
            moving_average_200_20 = df["SMA_200"].iloc[-20]
        except Exception:
            moving_average_200_20 = 0

        # ✅ Minervini Conditions
        condition_1 = currentClose > moving_average_150 > moving_average_200
        condition_2 = moving_average_150 > moving_average_200
        condition_3 = moving_average_200 > moving_average_200_20
        condition_4 = moving_average_50 > moving_average_150 > moving_average_200
        condition_5 = currentClose > moving_average_50
        condition_6 = currentClose >= (1.3 * low_of_52week)
        condition_7 = currentClose >= (0.75 * high_of_52week)

        if not all([condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7]):
            failed_conds = [
                f"1" if not condition_1 else "",
                f"2" if not condition_2 else "",
                f"3" if not condition_3 else "",
                f"4" if not condition_4 else "",
                f"5" if not condition_5 else "",
                f"6" if not condition_6 else "",
                f"7" if not condition_7 else "",
            ]
            reason = f"❌ Failed Minervini conditions: {', '.join(filter(None, failed_conds))}"
            
            detailedExportList = pd.concat([detailedExportList, pd.DataFrame([{
                'Stock': stock, 
                "RS_Rating": RS_Rating,
                "50 Day MA": moving_average_50, 
                "150 Day Ma": moving_average_150, 
                "200 Day MA": moving_average_200, 
                "52 Week Low": low_of_52week, 
                "52 week High": high_of_52week,
                "Status": reason
            }])], ignore_index=True)
            print(reason)

            print(f"✅ {stock} made the Minervini requirements")

    except Exception as e:
        print(f"⚠️ Could not process {stock}: {e}")

# ✅ Split the results
passed_df = detailedExportList[detailedExportList["Status"] == "✅ Passed Minervini"]
failed_df = detailedExportList[detailedExportList["Status"] != "✅ Passed Minervini"]

# 📤 Export each to a separate Excel file
passed_df.to_excel("Minervini_Passed.xlsx", index=False)
failed_df.to_excel("Minervini_Failed.xlsx", index=False)

C:\Users\930140\AppData\Local\Temp\1/ipykernel_32196/1393253152.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  detailedExportList = pd.concat([detailedExportList, pd.DataFrame([{


❌ Failed Minervini conditions: 1, 2, 3, 4, 6, 7
✅ AOS made the Minervini requirements
❌ Failed Minervini conditions: 5, 6
✅ ABT made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 4, 5, 6
✅ ABBV made the Minervini requirements
❌ Failed Minervini conditions: 1, 3, 4, 6
✅ ACN made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6, 7
✅ ADBE made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 7
✅ AMD made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 7
✅ AES made the Minervini requirements
❌ Failed Minervini conditions: 1, 5, 6
✅ AFL made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ A made the Minervini requirements
❌ Failed Minervini conditions: 1, 4, 5, 6
✅ APD made the Minervini requirements
❌ Failed Minervini conditions: 3, 4
✅ ABNB made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ AKAM made the Minervini requirements
❌ Failed Miner

❌ Failed Minervini conditions: 1, 2, 4, 5, 6
✅ KO made the Minervini requirements
❌ Failed Minervini conditions: 4, 6
✅ CTSH made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6
✅ CL made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6
✅ CMCSA made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ CAG made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ COP made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 4, 5, 6
✅ ED made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6, 7
✅ STZ made the Minervini requirements
❌ Failed Minervini conditions: 4
✅ CEG made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ COO made the Minervini requirements
❌ Failed Minervini conditions: 6
✅ CPRT made the Minervini requirements
❌ Failed Minervini conditions: 1, 4
✅ GLW made the Minervini requirements
❌ Failed Minervini

❌ Failed Minervini conditions: 1, 3, 4, 7
✅ HPE made the Minervini requirements
❌ Failed Minervini conditions: 4, 6
✅ HLT made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ HOLX made the Minervini requirements
❌ Failed Minervini conditions: 1, 4, 6
✅ HD made the Minervini requirements
❌ Failed Minervini conditions: 3, 4, 6
✅ HON made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6
✅ HRL made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6
✅ HST made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 7
✅ HPQ made the Minervini requirements
❌ Failed Minervini conditions: 1, 3, 4, 6
✅ HUBB made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ HUM made the Minervini requirements
❌ Failed Minervini conditions: 4
✅ HBAN made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4
✅ HII made the Minervini requirements
❌ Failed Minervini conditions

❌ Failed Minervini conditions: 1, 2, 4, 6
✅ NSC made the Minervini requirements
❌ Failed Minervini conditions: 4
✅ NTRS made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 4, 5, 6
✅ NOC made the Minervini requirements
❌ Failed Minervini conditions: 1, 3, 4, 7
✅ NCLH made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ NUE made the Minervini requirements
❌ Failed Minervini conditions: 3, 4
✅ NVDA made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ NVR made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 7
✅ NXPI made the Minervini requirements
❌ Failed Minervini conditions: 5
✅ ORLY made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6, 7
✅ OXY made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6, 7
✅ ODFL made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ OMC made the Minervini requirements
❌ Failed Mine

❌ Failed Minervini conditions: 1, 2, 3, 4, 6
✅ UNP made the Minervini requirements
❌ Failed Minervini conditions: 1, 4, 7
✅ UAL made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ UPS made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4
✅ URI made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ UNH made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6
✅ UHS made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4
✅ VLO made the Minervini requirements
❌ Failed Minervini conditions: 1, 5
✅ VTR made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 6
✅ VLTO made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 4, 5, 6
✅ VZ made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6
✅ VRTX made the Minervini requirements
❌ Failed Minervini conditions: 1, 2, 3, 4, 5, 6, 7
✅ VTRS made the Minervini requirements
❌ 